# Imports

In [1]:
# Basics
import pandas as pd
import numpy as np
np.random.seed(0)
from matplotlib import pyplot as plt
import seaborn as sns

# Job database wrapper
from JobsDb import JobsDb

# Text preprocessing
from keras.preprocessing import text, sequence

# Clustering tools 
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import calinski_harabasz_score

# Vectorizing Tools
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Classification Tools
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, GlobalMaxPool1D

# Pipeline tools
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer, OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier

# Model evaluation
from sklearn.metrics import confusion_matrix

# Load Data

In [2]:
def load_data():
    db = JobsDb()
    df = db.load_table_as_df('jobs')
    db.close()
    return df

df = load_data()
#df = df.iloc[9800:]
df.head()

,id,title,url,description
0,1,Psychiatrist (Per Diem) - #201222-4667HD,https://www.careerjet.com/jobad/us61b2e1c18a4d...,\n \n shall strive to be a global leader i...
1,2,Professor of Osteopathic Manipulative Medicine,https://www.careerjet.com/jobad/us0f769e5a939c...,\nPosition Details Position Title Professor o...
2,3,Amazon Warehouse Assistant - Immediately Hiring,https://www.careerjet.com/jobad/usba95a3670331...,\n \n Hourly pay rate: $15.00 *The base pa...
3,4,Non-QM Underwriter,https://www.careerjet.com/jobad/usb59eda9438ed...,\nJob Description We looking to add an experi...
4,5,"project manager, accounting systems",https://www.careerjet.com/jobad/us71cfd31d23d6...,\njob description job summary: Project Manag...


# Cluster Analysis of Titles

## Extract Titles and Descriptions 

In [3]:
def extract_data(df):
    titles = df['title']
    descriptions = df['description']
    return titles, descriptions

titles, descriptions = extract_data(df)

## Compute Title Vocabulary

In [4]:
def get_vocabulary(titles):
    vocabulary = set(word for title in titles for word in title.split(' '))
    return vocabulary

title_vocabulary = get_vocabulary(titles)
description_vocabulary = get_vocabulary(descriptions)

## Vectorize Vocabulary

In [5]:
def vectorize_vocabulary(vocabulary):
    glove = {}
    with open('../model/glove.6B.50d.txt', 'rb') as f:
        for line in f:
            parts = line.split()
            word = parts[0].decode('utf-8')
            if word in vocabulary:
                vector = np.array(parts[1:], dtype=np.float32)
                glove[word] = vector
    return glove

title_glove = vectorize_vocabulary(title_vocabulary)
description_glove = vectorize_vocabulary(description_vocabulary)

## Title Vectorizer Class

In [6]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(w2v))])
    
    # Note: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])
    
title_vectorizer = W2vVectorizer(title_glove)

## Build Vector Clustering Pipeline

In [7]:
cluster_pipe =  Pipeline([
    ('Word2Vec_Vectorizer', title_vectorizer),
    ('KMeans', KMeans(n_clusters=7))])

## Fit Pipeline on Titles

In [8]:
cluster_pipe.fit(titles)

Pipeline(steps=[('Word2Vec_Vectorizer',
                 <__main__.W2vVectorizer object at 0x7f58d1610a58>),
                ('KMeans', KMeans(n_clusters=7))])

## Build Target Classes

In [9]:
target = cluster_pipe.predict(titles)

# Deep NLP Model to Classify Descriptions 

## Word Vectorizer for Descriptions

In [10]:
model_folder = '/home/schart/Flatiron/DataScience/Phase5/Assessments/JobDash/model/'
glove_file = datapath(model_folder +'glove.6B.50d.txt')
tmp_file = get_tmpfile(model_folder+'glove_to_w2vec.txt')
_ = glove2word2vec(glove_file, tmp_file)
model_glove = KeyedVectors.load_word2vec_format(tmp_file)

## Preprocessor for Descriptions 

In [11]:
def preprocess_descriptions(X):
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(list(X))
    list_tokenized_descriptions = tokenizer.texts_to_sequences(X)
    X = sequence.pad_sequences(list_tokenized_descriptions, maxlen=500)
    return X

preprocessor = FunctionTransformer(preprocess_descriptions)

## Build Classifier

In [12]:
def build_classifier():
    classifier = Sequential()
    embedding_size = 128
    #classifier.add(Embedding(20000, embedding_size))
    classifier.add(model_glove.get_keras_embedding())
    classifier.add(LSTM(50, return_sequences=False))
    classifier.add(Dense(50, activation='relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(7, activation='softmax'))
    classifier.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )   
    return classifier

classifier = KerasClassifier(
    build_fn=build_classifier, 
    epochs=5, 
    batch_size=32, 
    validation_split=0.1,
    workers=4
)

## Build Classifier Pipeline

In [13]:
classifier_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ]
)

## Fit Classifier Pipeline

In [ ]:
classifier_pipe.fit(descriptions, target)

Epoch 1/5
539/539 [==============================] - 413s 766ms/step - loss: 1.2913 - accuracy: 0.5175 - val_loss: 1.1529 - val_accuracy: 0.5931
Epoch 2/5
539/539 [==============================] - 420s 780ms/step - loss: 1.0783 - accuracy: 0.5836 - val_loss: 1.1421 - val_accuracy: 0.5952
Epoch 3/5
539/539 [==============================] - 422s 782ms/step - loss: 1.0366 - accuracy: 0.6012 - val_loss: 1.1133 - val_accuracy: 0.6004
Epoch 4/5
539/539 [==============================] - 422s 783ms/step - loss: 1.0102 - accuracy: 0.6112 - val_loss: 1.1910 - val_accuracy: 0.5592
Epoch 5/5
280/539 [==============>...............] - ETA: 11:33 - loss: 0.9901 - accuracy: 0.6210- ETA: 10:23 - loss: 

# Make Predictions

In [ ]:
pred = classifier_pipe.predict(descriptions)

# Confusion Matrix

In [ ]:
confmat = confusion_matrix(target, pred)

In [ ]:
sns.heatmap(confmat, cmap='Blues')